In [1]:
from coffea.nanoevents import BaseSchema, NanoEventsFactory
import dask_awkward as dak
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import hist
import mplhep as hep
import numba
import vector
vector.register_awkward()

In [2]:
# events = NanoEventsFactory.from_root(
#     file="./events_2021100911011863214.root:events",
#     schemaclass=BaseSchema,
#     # schemaclass=DelphesSchema, Doesn't work 
#     # schemaclass=NanoAODSchema.v7, Doesn't work
#     metadata={
#         "path":"/eos/experiment/fcc/ee/generation/DelphesEvents/spring2021/IDEA/kkmcp8_ee_mumu_ecm240/",
#         "name":"kkmcp8_ee_mumu_ecm240",
#         "process":"Z/gamma* to mumu, ecm=240 GeV"
#     }
# ).events()

In [3]:
events = NanoEventsFactory.from_root(
    file="./p8_ee_ZH_ecm240/events_101027117.root:events",
    schemaclass=BaseSchema,
    # schemaclass=DelphesSchema, Doesn't work 
    # schemaclass=NanoAODSchema.v7, Doesn't work
    metadata={
        "path":"/eos/experiment/fcc/ee/generation/DelphesEvents/spring2021/IDEA/p8_ee_ZH_ecm240/events_101027117.root",
        "name":"p8_ee_ZH_ecm240",
        "process":"ZH to mumu, ecm=240 GeV"
    }
).events()

In [4]:
events.fields

['Electron',
 'Electron#0',
 'Electron#0/Electron#0.index',
 'Electron#0/Electron#0.collectionID',
 'Muon',
 'Muon#0',
 'Muon#0/Muon#0.index',
 'Muon#0/Muon#0.collectionID',
 'AllMuon',
 'AllMuon#0',
 'AllMuon#0/AllMuon#0.index',
 'AllMuon#0/AllMuon#0.collectionID',
 'EFlowNeutralHadron',
 'EFlowNeutralHadron/EFlowNeutralHadron.type',
 'EFlowNeutralHadron/EFlowNeutralHadron.energy',
 'EFlowNeutralHadron/EFlowNeutralHadron.energyError',
 'EFlowNeutralHadron/EFlowNeutralHadron.position.x',
 'EFlowNeutralHadron/EFlowNeutralHadron.position.y',
 'EFlowNeutralHadron/EFlowNeutralHadron.position.z',
 'EFlowNeutralHadron/EFlowNeutralHadron.positionError[6]',
 'EFlowNeutralHadron/EFlowNeutralHadron.iTheta',
 'EFlowNeutralHadron/EFlowNeutralHadron.phi',
 'EFlowNeutralHadron/EFlowNeutralHadron.directionError.x',
 'EFlowNeutralHadron/EFlowNeutralHadron.directionError.y',
 'EFlowNeutralHadron/EFlowNeutralHadron.directionError.z',
 'EFlowNeutralHadron/EFlowNeutralHadron.shapeParameters_begin',
 'EFlo

In [5]:
mu_0_index = events['Muon#0/Muon#0.index'].compute()

In [6]:
Recon = events['ReconstructedParticles/ReconstructedParticles.energy'].compute()

In [7]:
useful_events = events[ak.num(Recon) > 0]

In [8]:
useful_events['ReconstructedParticles/ReconstructedParticles.energy'].compute()

<Array [[0.388, 1.19, ..., 2.9, 1.22], ...] type='99984 * [var * float32[pa...'>

In [9]:
Reco_E = useful_events['ReconstructedParticles/ReconstructedParticles.energy'].compute()
Muon_index = useful_events['Muon#0/Muon#0.index'].compute()

In [10]:
Reco_E[:,[0]]

<Array [[0.388], [0.305], ..., [0.539], [30.6]] type='99984 * 1 * float32[p...'>

In [11]:
a = ak.Array([
    [23.5, 34.9],
    [45.6,78.0,86.7],
    [36.0]
])
idx = ak.Array([
    [0],
    [1,2],
    []
])

In [12]:
a[0]

<Array [23.5, 34.9] type='2 * float64'>

In [13]:
def index_mask(input_array, index_array):
    if len(input_array) != len(index_array) :
        raise Exception('Length of Input_array and index_array does not match!')
    counts = len(ak.count(input_array, axis = 1))
    @numba.jit
    def numba_wrap(input_array, index_array,counts):
        output_array = []
        for event_index in range(counts):
            event_mask = index_array[event_index]
            reco_list = input_array[event_index]
            output_array.append([reco_list[i] for i in  event_mask])
        return output_array
    return ak.Array(numba_wrap(input_array,index_array,counts))

In [14]:
index_mask(a,idx)

<Array [[23.5], [78, 86.7], []] type='3 * var * float64'>

In [15]:
Muon_E = index_mask(Reco_E, Muon_index)

In [16]:
Muon_E[ak.num(Muon_E)>0]

<Array [[43.4], [61.6, 41.5], ..., [29.8], [36.6]] type='11229 * var * float64'>